<a href="https://colab.research.google.com/github/karino2/tegashiki/blob/master/TFLite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# !pip install tensorflow==2.0.0-beta0

In [0]:
# !pip install tf-nightly

In [0]:
# today tensorflow seems update and batchnormalization name scope is changed. I downgrade for a while
# !pip install tensorflow==1.13.1

In [0]:
import tensorflow as tf

In [0]:
# from tensorflow.contrib import lite

In [2]:
print(tf.VERSION)
# print(tf.__version__)

1.14.0


In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
# tf.keras.backend.clear_session()
# tf.keras.backend.set_learning_phase(0) 

In [0]:
# MODEL_DIR="gs://karino2-tegashiki/models/exports/export_expgen_rnn_small_dropout05_fixshape/1560480405/"
# MODEL_DIR="gs://karino2-tegashiki/models/exports/export_expgen_rnn_small_dropout05_trainflaghandle/1560556738/"
# MODEL_DIR="gs://karino2-tegashiki/models/exports/export_expgen_convencdec_posfc/1560866662"
# MODEL_DIR="gs://karino2-tegashiki/models/exports/expgen_convencdec_myembed/1560903469"
# MODEL_DIR="gs://karino2-tegashiki/models/exports/expgen_convencdec_myembed2/1560910035"
# MODEL_DIR="gs://karino2-tegashiki/models/exports/expgen_convencdec_myembed2/1560933034"
# MODEL_DIR="gs://karino2-tegashiki/models/exports/expgen_convencdec_myembed2/1560943781"
# MODEL_DIR="gs://karino2-tegashiki/models/exports/expgen_convencdec_myembed_small/1560974897"
#MODEL_DIR="gs://karino2-tegashiki/models/exports/expgen_convencdec_fixfutureleak/1561197058"
# MODEL_DIR="gs://karino2-tegashiki/models/exports/expgen_convencdec_enc256x13dec128x5/1561326828"
# MODEL_DIR="gs://karino2-tegashiki/models/exports/expgen_plain_dechidden256_fefix/1562225530"

MODEL_DIR="gs://karino2-tegashiki/models/exports/expgen_plain_encdecatten_pat2_trainopfix/1563243864"

In [0]:
LITE = tf.lite
# LITE = lite

In [0]:
# !gsutil ls {MODLE_DIR}

### Convert to TFLite

In [0]:
MAX_STROKE_NUM=22
MAX_ONE_STROKE_LEN=50
INPUT_TYPE_DIM=3

In [8]:
# converter = LITE.TFLiteConverter.from_saved_model(MODEL_DIR, input_shapes={"stroke_input" : [MAX_STROKE_NUM, MAX_ONE_STROKE_LEN, INPUT_TYPE_DIM]})
converter = LITE.TFLiteConverter.from_saved_model(MODEL_DIR)
# workaround https://github.com/tensorflow/tensorflow/issues/26413
# converter.post_training_quantize=True
converter.optimizations = [LITE.Optimize.OPTIMIZE_FOR_SIZE]

tflite_quant_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_quant_model)

W0716 02:30:01.340767 140673038223232 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/lite/python/convert_saved_model.py:60: load (from tensorflow.python.saved_model.loader_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
W0716 02:30:03.082570 140673038223232 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
W0716 02:30:08.209443 140673038223232 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/lite/python/util.

22985992

In [0]:
# TFLITE_MODEL_NAME="convencdec_enc256x13dec128x5.tflite"
# TFLITE_MODEL_NAME="plain_dechidden256_fefix.tflite"

TFLITE_MODEL_NAME="plain_encdecatten_pat2_trainopfix.tflite"

In [10]:
!cp converted_model.tflite {TFLITE_MODEL_NAME}
!gsutil cp {TFLITE_MODEL_NAME} gs://karino2-tegashiki/models/mobile/

Copying file://plain_encdecatten_pat2_trainopfix.tflite [Content-Type=application/octet-stream]...
-
Operation completed over 1 objects/21.9 MiB.                                     


In [0]:
!gsutil ls gs://karino2-tegashiki/models/mobile/

gs://karino2-tegashiki/models/mobile/convencdec_enc256x13dec128x5.tflite
gs://karino2-tegashiki/models/mobile/convencdec_fixfutureleak.tflite
gs://karino2-tegashiki/models/mobile/convencdec_myembed2.tflite
gs://karino2-tegashiki/models/mobile/convencdec_myembed_small.tflite


### Test

In [0]:
import numpy as np
import tensorflow as tf

In [0]:
BEGIN_OF_SEQ = 113
MAX_STROKE_NUM=22
MAX_TOKEN_LEN=10+2


In [0]:
# MODEL_FILE="convencdec_myembed2.tflite"
MODEL_FILE="converted_model.tflite"
interpreter = tf.lite.Interpreter(model_path=MODEL_FILE)

In [0]:
interpreter.allocate_tensors()

In [14]:
interpreter.get_input_details()

[{'dtype': numpy.int32,
  'index': 209,
  'name': 'input_decoder',
  'quantization': (0.0, 0),
  'shape': array([ 1, 12], dtype=int32)},
 {'dtype': numpy.int32,
  'index': 213,
  'name': 'stroke_input',
  'quantization': (0.0, 0),
  'shape': array([ 1, 22, 50,  3], dtype=int32)}]

In [15]:
interpreter.get_output_details()

[{'dtype': numpy.float32,
  'index': 226,
  'name': 'time_distributed/Reshape_1',
  'quantization': (0.0, 0),
  'shape': array([  1,  12, 115], dtype=int32)}]

In [0]:
DECODER_INPUT_ID=209
STROKE_INPUT_ID=213
OUTPUT_ID=226

In [0]:
def predict(interpreter, inputst, inputdec):
  
  interpreter.set_tensor(DECODER_INPUT_ID, inputdec.astype(np.int32).reshape([1, MAX_TOKEN_LEN]))
  interpreter.set_tensor(STROKE_INPUT_ID, inputst.astype(np.int32).reshape([1, MAX_STROKE_NUM, 50, 3]))
  interpreter.invoke()
  inference_res = interpreter.get_tensor(OUTPUT_ID)
  return [inference_res[0, i, :].argmax() for i in range(MAX_TOKEN_LEN)]

In [0]:
import json

In [0]:
JSON_FILE="20190716_12013337.json"

In [0]:
with open(JSON_FILE) as f:
  data = json.load(f)

In [0]:
normalized = np.array(data[1]).reshape(MAX_STROKE_NUM, 50, 3)

In [32]:
predict(interpreter, normalized,  np.array([BEGIN_OF_SEQ, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

[37, 75, 34, 34, 34]

In [33]:
predict(interpreter, normalized,  np.array([BEGIN_OF_SEQ, 37, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

[37, 29, 34, 34, 34]

In [35]:
predict(interpreter, normalized,  np.array([BEGIN_OF_SEQ, 37, 29, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

[37, 29, 34, 34, 34, 34, 34, 34, 47, 114, 114, 114]

In [36]:
predict(interpreter, normalized,  np.array([BEGIN_OF_SEQ, 37, 29, 34, 0, 0, 0, 0, 0, 0, 0, 0]))

[37, 29, 34, 109, 47, 114, 114, 114, 114, 114, 114, 114]

In [37]:
predict(interpreter, normalized,  np.array([BEGIN_OF_SEQ, 37, 29, 34, 109, 0, 0, 0, 0, 0, 0, 0]))

[37, 29, 34, 109, 114, 114, 114, 114, 114, 114, 114, 114]

In [41]:
predict(interpreter, normalized,  np.array([BEGIN_OF_SEQ, 37, 31, 47, 54, 0, 0, 0, 0, 0, 0, 0]))

[37, 29, 34, 102, 34, 34, 34, 34, 47, 114, 114, 114]

In [42]:
predict(interpreter, normalized, np.array([113, 37, 29, 47, 52, 62, 52, 114, 0, 0, 0, 0]))

[37, 29, 47, 52, 89, 29, 114, 0, 114, 80, 80, 35]

In [39]:
!ls -l converted_model.tflite

-rw-r--r-- 1 root root 22985992 Jul 16 02:30 converted_model.tflite


In [0]:
decoder_input = np.array([112,  76,  75,  76, 113], dtype=np.int32)
stroke_input = np.array([[[94, 1155, 1], [147, 1145, 1], [207, 1151, 1], [273, 1179, 1], [340, 1218, 1], [407, 1275, 1], [467, 1349, 1], [510, 1425, 1], [543, 1508, 1], [564, 1588, 1], [573, 1667, 1], [567, 1738, 1], [550, 1798, 1], [530, 1848, 1], [501, 1895, 1], [467, 1928, 1], [427, 1961, 1], [384, 1981, 1], [340, 1998, 1], [297, 2004, 1], [257, 2008, 1], [221, 2008, 1], [188, 2004, 1], [157, 2000, 1], [134, 1991, 1], [114, 1987, 1], [101, 1974, 1], [94, 1967, 1], [88, 1951, 1], [84, 1941, 1], [88, 1928, 1], [84, 1908, 1], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]], [[7, 263, 1], [11, 276, 1], [14, 319, 1], [18, 382, 1], [21, 472, 1], [24, 579, 1], [24, 709, 1], [27, 842, 1], [27, 981, 1], [24, 1114, 1], [21, 1238, 1], [21, 1349, 1], [18, 1451, 1], [14, 1534, 1], [11, 1608, 1], [7, 1667, 1], [7, 1715, 1], [11, 1754, 1], [14, 1782, 1], [21, 1788, 1], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]], [[681, 8, 1], [667, 26, 1], [615, 520, 1], [615, 516, 1], [615, 511, 1], [615, 492, 1], [615, 483, 1], [620, 446, 1], [629, 427, 1], [653, 395, 1], [662, 381, 1], [699, 353, 1], [709, 348, 1], [755, 348, 1], [765, 348, 1], [806, 353, 1], [816, 357, 1], [839, 390, 1], [839, 404, 1], [839, 436, 1], [834, 450, 1], [802, 483, 1], [788, 492, 1], [741, 516, 1], [727, 520, 1], [681, 530, 1], [667, 530, 1], [643, 525, 1], [639, 520, 1], [629, 506, 1], [629, 502, 1], [629, 497, 1], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]]],
                       dtype=np.int32)

In [0]:
interpreter.set_tensor(136, decoder_input.reshape([1, 5]))
interpreter.set_tensor(140, stroke_input.reshape([1, 13, 50, 3]))

In [0]:
interpreter.invoke()

In [0]:
inference_res = interpreter.get_tensor(153)

In [0]:
inference_res.shape

(1, 5, 114)

In [0]:
[inference_res[0, i, :].argmax() for i in range(5)]

[76, 76, 76, 113, 0]

- inference_res: bbb
- label b^b

In [0]:
DATA_DIR="gs://karino2-tegashiki/dataset"
TF_RECORD_FILE="{}/crohme2019_onesupsubtwo_train.tfrecord.gz".format(DATA_DIR)
TF_VALID_RECORD_FILE="{}/crohme2019_onesupsubtwo_valid.tfrecord.gz".format(DATA_DIR)

In [0]:
MAX_TOKEN_LEN=3+2

In [0]:
def parser(serialized_example):
  featurelen = MAX_STROKE_NUM*MAX_ONE_STROKE_LEN
  
  features = tf.parse_single_example(
      serialized_example,
      features={
      'input_x': tf.FixedLenFeature([featurelen], tf.int64),
      'input_y': tf.FixedLenFeature([featurelen], tf.int64),
      'input_type': tf.FixedLenFeature([featurelen], tf.int64),
      'decoder_input':tf.FixedLenFeature([MAX_TOKEN_LEN], tf.int64),
      'decoder_labels':tf.FixedLenFeature([MAX_TOKEN_LEN], tf.int64)}          
  )
  
  input_x, input_y, input_type = [tf.reshape(tf.cast(features[fname], tf.int32), [MAX_STROKE_NUM, MAX_ONE_STROKE_LEN]) for fname in ['input_x', 'input_y', 'input_type']]
  one_sample_stroke = tf.stack([input_x, input_y, input_type], 2)
  decoder_input = tf.cast(features["decoder_input"], tf.int32)
  decoder_labels = tf.cast(features["decoder_labels"], tf.int32)
  
  return {"input_stroke": one_sample_stroke, "input_decoder": decoder_input} , decoder_labels

In [0]:
dataset = tf.data.TFRecordDataset(TF_RECORD_FILE, "GZIP")
dataset = dataset.map(parser)

In [0]:
sess = tf.InteractiveSession()

In [0]:
res = []
iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()

for i in range(100):
  res.append(sess.run(next_element))

W0623 22:32:47.301916 140320814720896 deprecation.py:323] From <ipython-input-29-8879d7b4ade6>:2: DatasetV1.make_one_shot_iterator (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


[109, 79, 113, 113, 0]

In [0]:
stroke = np.array([[[133, 444, 1],
  [174, 362, 1],
  [174, 321, 1],
  [174, 280, 1],
  [338, 979, 1],
  [380, 1021, 1],
  [462, 1062, 1],
  [544, 1062, 1],
  [627, 1062, 1],
  [709, 979, 1],
  [750, 938, 1],
  [832, 856, 1],
  [874, 774, 1],
  [874, 691, 1],
  [874, 609, 1],
  [874, 527, 1],
  [874, 485, 1],
  [874, 444, 1],
  [874, 403, 1],
  [874, 444, 1],
  [874, 527, 1],
  [874, 609, 1],
  [915, 691, 1],
  [915, 856, 1],
  [915, 979, 1],
  [915, 1144, 1],
  [915, 1350, 1],
  [874, 1514, 1],
  [791, 1638, 1],
  [750, 1761, 1],
  [627, 1885, 1],
  [503, 1967, 1],
  [421, 2008, 1],
  [297, 2008, 1],
  [174, 2008, 1],
  [91, 2008, 1],
  [50, 2008, 1],
  [9, 1967, 1],
  [9, 1926, 1],
  [9, 1885, 1],
  [9, 1844, 1],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0]],
 [[953, 35, 1],
  [953, 88, 1],
  [953, 168, 1],
  [953, 221, 1],
  [980, 327, 1],
  [980, 353, 1],
  [980, 380, 1],
  [980, 433, 1],
  [980, 459, 1],
  [980, 486, 1],
  [980, 513, 1],
  [980, 539, 1],
  [1006, 513, 1],
  [1006, 486, 1],
  [1033, 406, 1],
  [1033, 353, 1],
  [1033, 300, 1],
  [1059, 221, 1],
  [1086, 194, 1],
  [1086, 141, 1],
  [1112, 88, 1],
  [1192, 35, 1],
  [1245, 8, 1],
  [1272, 8, 1],
  [1298, 8, 1],
  [1325, 8, 1],
  [1351, 8, 1],
  [1378, 8, 1],
  [1404, 88, 1],
  [1404, 115, 1],
  [1404, 168, 1],
  [1404, 194, 1],
  [1404, 221, 1],
  [1404, 247, 1],
  [1404, 274, 1],
  [1404, 300, 1],
  [1404, 327, 1],
  [1404, 380, 1],
  [1404, 406, 1],
  [1404, 433, 1],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0]],
 [[0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0]],
 [[0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0]],
 [[0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0]],
 [[0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0]],
 [[0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0]],
 [[0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0]],
 [[0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0]],
 [[0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0]],
 [[0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0]],
 [[0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0]],
 [[0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0]]])

In [0]:
predict(interpreter, stroke, np.array([112, 0, 0, 0, 0]))

[109, 79, 113, 113, 0]

In [0]:
predict(interpreter, stroke, np.array([112, 109, 0, 0, 0]))

[109, 75, 113, 113, 0]

In [0]:
predict(interpreter, stroke, np.array([112, 109, 75, 0, 0]))

[109, 75, 79, 113, 0]

In [0]:
predict(interpreter, stroke, np.array([112, 109, 75, 79, 0]))

[109, 75, 79, 113, 0]

In [0]:
interpreter.set_tensor(136, res[4][0]['input_decoder'].reshape([1, 5]))
interpreter.set_tensor(140, res[4][0]['input_stroke'].reshape([1, 13, 50, 3]))

In [0]:
interpreter.invoke()

In [0]:
inference_res = interpreter.get_tensor(153)

In [0]:
[inference_res[0, i, :].argmax() for i in range(5)]

[35, 75, 10, 113, 0]

In [0]:
interpreter.set_tensor(136, np.array([112, 0, 0, 0, 0]).astype(np.int32).reshape([1, 5]))
interpreter.invoke()
inference_res = interpreter.get_tensor(153)
[inference_res[0, i, :].argmax() for i in range(5)]


[109, 79, 113, 113, 0]

In [0]:
interpreter.set_tensor(136, np.array([112, 109, 0, 0, 0]).astype(np.int32).reshape([1, 5]))
interpreter.invoke()
inference_res = interpreter.get_tensor(153)
[inference_res[0, i, :].argmax() for i in range(5)]


[109, 75, 113, 113, 0]

In [0]:
res[4][0]['input_decoder']

array([112,  35,  75,  80, 113], dtype=int32)

In [0]:
import json

In [0]:
strokelst = res[0][0]['input_stroke'].tolist()

In [0]:
len(strokelst)

13

In [0]:
with open("stroke.json", 'w', encoding='utf-8') as f:
  json.dump(strokelst, f)

In [0]:
def get_if_necessary(fname):
  if os.path.exists(fname):
    return
  !gsutil cp gs://karino2-tegashiki/dataset/{fname} {fname}

def get_and_load(fname):
  get_if_necessary(fname)
  with gzip.open(fname,'rb') as f:
    return pickle.load(f)  
  
def send_file(fname):
  !gsutil cp {fname} gs://karino2-tegashiki/dataset/
    
def dump_and_send(obj, fname):
  with gzip.open(fname,'wb') as f:
    pickle.dump(obj, f)
  send_file(fname)

### Misc

In [0]:
import numpy as np

In [0]:
stroke_input = np.array([[[207, 1151, 1], [273, 1179, 1], [340, 1218, 1], [407, 1275, 1], [467, 1349, 1], [510, 1425, 1], [543, 1508, 1], [564, 1588, 1], [573, 1667, 1], [567, 1738, 1], [550, 1798, 1], [530, 1848, 1], [501, 1895, 1], [467, 1928, 1], [427, 1961, 1], [384, 1981, 1], [340, 1998, 1], [297, 2004, 1], [257, 2008, 1], [221, 2008, 1], [188, 2004, 1], [157, 2000, 1], [134, 1991, 1], [114, 1987, 1], [101, 1974, 1], [94, 1967, 1], [88, 1951, 1], [84, 1941, 1], [88, 1928, 1], [84, 1908, 1], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],
                         [[7, 263, 1], [11, 276, 1], [14, 319, 1], [18, 382, 1], [21, 472, 1], [24, 579, 1], [24, 709, 1], [27, 842, 1], [27, 981, 1], [24, 1114, 1], [21, 1238, 1], [21, 1349, 1], [18, 1451, 1], [14, 1534, 1], [11, 1608, 1], [7, 1667, 1], [7, 1715, 1], [11, 1754, 1], [14, 1782, 1], [21, 1788, 1], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]], 
                         [[681, 8, 1], [667, 26, 1], [615, 520, 1], [615, 516, 1], [615, 511, 1], [615, 492, 1], [615, 483, 1], [620, 446, 1], [629, 427, 1], [653, 395, 1], [662, 381, 1], [699, 353, 1], [709, 348, 1], [755, 348, 1], [765, 348, 1], [806, 353, 1], [816, 357, 1], [839, 390, 1], [839, 404, 1], [839, 436, 1], [834, 450, 1], [802, 483, 1], [788, 492, 1], [741, 516, 1], [727, 520, 1], [681, 530, 1], [667, 530, 1], [643, 525, 1], [639, 520, 1], [629, 506, 1], [629, 502, 1], [629, 497, 1], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]], 
                         [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],
                         [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],
                         [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],
                         [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],
                         [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]], 
                         [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],
                         [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],
                         [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],
                         [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],
                         [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]]])

In [0]:
len(stroke_input[0])

48

In [0]:
len(stroke_input[3])

50

In [0]:
stroke_input

In [0]:
tmp = stroke_input[:, :, 2] == 1

In [0]:
tmp.shape

(13, 50)

In [0]:
stroke_input[tmp].shape

(84, 3)

In [0]:
stroke_input.shape

(13, 50, 3)

In [0]:
tmp.dtype

dtype('bool')

In [0]:
import numpy as np

In [0]:
np.array(range(15)).reshape([3, 5])

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14]])

In [0]:
res = np.arange(10).reshape([2, 5])

In [0]:
res[1, 4]

9

In [0]:
res = np.array(range(15)).reshape([3, 5])

In [0]:
res[0:2, 1:4]

array([[1, 2, 3],
       [6, 7, 8]])

In [0]:
res[1, 2:4]

array([7, 8])

In [0]:
res.shape

(3, 5)

In [0]:
res[2, 4]

14

In [0]:
res = np.arange(30).reshape([2, 3, 5])

In [0]:
res.shape

(2, 3, 5)

In [0]:
res[0]

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14]])

In [0]:
res[1, 0:2, 1:4]

array([[16, 17, 18],
       [21, 22, 23]])

In [0]:
!ls -l *.tflite

-rw-r--r-- 1 root root 357832 Jun 19 08:33 convencdec_myembed2.tflite


In [0]:
patch_file = "/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/graph_util_impl.py"

In [0]:
import os

In [0]:
org_file = patch_file + "_org"

In [0]:
if not os.path.exists(org_file):
  !cp {patch_file} {org_file}

In [0]:
%pycat {patch_file}

In [0]:
%%writefile {patch_file}


In [0]:
%%writefile {patch_file}
rewrite content here

Overwriting /usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/graph_util_impl.py


In [0]:
patch_fname = os.path.basename(patch_file)

In [0]:
!cp {patch_file} ./{patch_fname}

In [0]:
!echo ./{patch_fname}

./graph_util_impl.py


In [0]:
%%writefile {temp}

Hello, Hy This is test.
"""triple quote here."""

Overwriting test.txt


In [0]:
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()
open("model.tflite", "wb").write(tflite_model)
#converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
converter.post_training_quantize=True
tflite_quantized_model=converter.convert()
open("quantized_model.tflite", "wb").write(tflite_quantized_model)

In [0]:
!gsutil ls {MODEL_DIR}

gs://karino2-tegashiki/models/exports/export_expgen_rnn_small_dropout05_fixshape/
gs://karino2-tegashiki/models/exports/export_expgen_rnn_small_dropout05_fixshape/1560480405/
gs://karino2-tegashiki/models/exports/export_expgen_rnn_small_dropout05_fixshape/temp-b'1560480340'/
gs://karino2-tegashiki/models/exports/export_expgen_rnn_small_dropout05_fixshape/temp-b'1560480363'/
